In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import src.enformer_convs as enformer_convs

import training_utils_PEAKS as training_utils

from scipy import stats

2022-12-17 03:26:48.264763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-17 03:26:48.264810: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the

In [2]:
inits = training_utils.get_initializers("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights")
model = enformer_convs.enformer_convs(load_init=True,inits=inits)

2022-12-17 03:26:50.303732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-17 03:26:50.303780: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-17 03:26:50.303802: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist
2022-12-17 03:26:50.304089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
list_files = (tf.io.gfile.glob(os.path.join("gs://picard-testing-176520/enformer_baseline_ATAC_peaks_genomewide",
                                            "*.tfr")))

In [4]:
def one_hot(sequence):
    '''
    convert input string tensor to one hot encoded
    will replace all N character with 0 0 0 0
    '''
    vocabulary = tf.constant(['A', 'C', 'G', 'T'])
    mapping = tf.constant([0, 1, 2, 3])

    init = tf.lookup.KeyValueTensorInitializer(keys=vocabulary,
                                               values=mapping)
    table = tf.lookup.StaticHashTable(init, default_value=5) # makes N correspond to all 0s

    input_characters = tfs.upper(tfs.unicode_split(sequence, 'UTF-8'))

    out = tf.one_hot(table.lookup(input_characters), 
                      depth = 4, 
                      dtype=tf.float32)
    return out

def rev_comp_one_hot(sequence):
    '''
    convert input string tensor to one hot encoded
    will replace all N character with 0 0 0 0
    '''
    input_characters = tfs.upper(tfs.unicode_split(sequence, 'UTF-8'))
    input_characters = tf.reverse(input_characters,[0])
    
    vocabulary = tf.constant(['T', 'G', 'C', 'A'])
    mapping = tf.constant([0, 1, 2, 3])

    init = tf.lookup.KeyValueTensorInitializer(keys=vocabulary,
                                               values=mapping)
    table = tf.lookup.StaticHashTable(init, default_value=1)

    out = tf.one_hot(table.lookup(input_characters), 
                      depth = 4, 
                      dtype=tf.float32)
    return out

def random_encode(input_tuple):
    sequence, randint = input_tuple
    if randint == 0:
        return one_hot(sequence)
    else:
        return rev_comp_one_hot(sequence)

In [5]:
def deserialize(serialized_example,
                peaks_center_length,
                number_peaks):
    """
    Deserialize bytes stored in TFRecordFile.
    """
    feature_map = {
        'peaks_sequences': tf.io.FixedLenFeature([],tf.string)
    }
    
    data = tf.io.parse_example(serialized_example, feature_map)
    
    
    
    ### process peaks
    # first we want to randomly select the input peaks, let's say top 2000 out of 5000
    split_test=tf.strings.split(
        data['peaks_sequences'], sep='|', maxsplit=-1, name=None
    )
    split_test = split_test[:-1]
    
    idxs = tf.range(tf.shape(split_test)[0])
    ridxs = tf.random.shuffle(idxs)[:number_peaks]
    random_sample = tf.gather(split_test, ridxs)
    
    randints=tf.math.round(tf.random.uniform(shape=[number_peaks,],minval=0,maxval=1))
    
    peaks_sequences=tf.map_fn(random_encode,(random_sample,randints),fn_output_signature=tf.float32)
    
    return {
        'peaks_sequences': tf.ensure_shape(peaks_sequences,[number_peaks,peaks_center_length,4])
    }

In [ ]:
for ind_file in list_files:
    
    files = tf.data.Dataset.list_files(ind_file)
    
    dataset = tf.data.TFRecordDataset(files,
                                      compression_type='ZLIB',
                                      num_parallel_reads=1)
    
    dataset = dataset.map(lambda record: deserialize(record,
                                                     128,
                                                     10000),
                          deterministic=False,
                          num_parallel_calls=1)
    dataset= dataset.repeat(1).batch(1)
    
    out=next(iter(dataset))
    model_out=model(out['peaks_sequences'])
    mean=tf.reduce_mean(model_out[0,:,0,:],axis=0)
    
    name=ind_file.split('/')[-1].split('.')[0]
    np.savetxt('enformer_base_line_GAP_peaks/' + name + '.out', mean, delimiter=',')
    
    

In [8]:
mean=tf.reduce_mean(model_out[0,:,0,:],axis=0)

In [10]:
np.savetxt('test.out', mean, delimiter=',')

In [ ]:
len(unstacked)

In [13]:
np.genfromtxt("test.out", delimiter=",")

array([ 51.32778168, -29.50163078, 110.26683807, ...,  43.71905518,
         0.2550846 ,  36.57140732])

In [16]:
ind_file.split('/')[-1].split('.')[0]

'HG_A375'